In [1]:
import torch
from torchvision import datasets, transforms
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import time


# Step 1: Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Download and prepare dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


# Convert dataset to numpy arrays
def dataset_to_numpy(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size=len(train_dataset), shuffle=False)
    data, labels = next(iter(loader))
    return data[:].numpy(), labels[:].numpy()

X_train, y_train = dataset_to_numpy(train_dataset)
X_test, y_test = dataset_to_numpy(test_dataset)

# Step 2: Flatten Images
X_train = X_train.reshape(len(X_train), -1)  # Flatten to [n_samples, n_features]
X_test = X_test.reshape(len(X_test), -1)
print(X_train.shape)

# Step 3: Standardize the Data after we flattened the image , so all the values have the same range
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


from sklearn.decomposition import PCA
# Step -: Apply PCA to keep 90% of the variance
pca = PCA(n_components=0.9)  # Keep 90% of the variance
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Step 4: Train SVM Classifier with rbf kernel
rbf_start = time.time()
rbf_svm = SVC(kernel='rbf', C=1.0) 
rbf_svm.fit(X_train, y_train)
rbf_end = time.time()

# Step 5: Evaluate the Model
y_pred = rbf_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}% , took {rbf_end - rbf_start}")

# Step 6: Train SVM Classifier with linear kernel
linear_start = time.time()
linear_svm = SVC(kernel='linear', C=1.0, max_iter=10000)
linear_svm.fit(X_train, y_train)
linear_end = time.time()

# Step 7: Evaluate the Model
y_pred = linear_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}% , took {linear_end - linear_start}")


Files already downloaded and verified
Files already downloaded and verified
(50000, 3072)
Test Accuracy: 54.06% , took 267.64582443237305


c:\Users\jimnt\Desktop\dl-pytorch\.venv\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Test Accuracy: 14.99% , took 169.7345838546753


In [2]:
from sklearn.metrics import accuracy_score
import numpy as np


# Function to compute per-class accuracy
def per_class_accuracy(y_true, y_pred, num_classes):
    class_accuracies = {}
    for cls in range(num_classes):
        # Get indices where the true labels are equal to the current class
        cls_indices = (y_true == cls)
        # Compute accuracy for this class
        correct = np.sum(y_pred[cls_indices] == y_true[cls_indices])
        total = np.sum(cls_indices)
        class_accuracies[cls] = (correct / total) * 100 if total > 0 else 0
    return class_accuracies

# Step 5: Evaluate the RBF Kernel Model
y_pred_rbf = rbf_svm.predict(X_test)
overall_accuracy_rbf = accuracy_score(y_test, y_pred_rbf)

# Calculate per-class accuracy for RBF Kernel
class_accuracies_rbf = per_class_accuracy(y_test, y_pred_rbf, num_classes=10)

# Print per-class accuracies for RBF Kernel
print("RBF Kernel accuracy")
for cls, acc in class_accuracies_rbf.items():
    print(f"Class {cls}: {acc:.2f}%")

# Step 7: Evaluate the Linear Kernel Model
y_pred_linear = linear_svm.predict(X_test)
overall_accuracy_linear = accuracy_score(y_test, y_pred_linear)

# Calculate per-class accuracy for Linear Kernel
class_accuracies_linear = per_class_accuracy(y_test, y_pred_linear, num_classes=10)

# Print per-class accuracies for Linear Kernel
print("Linear Kernel accuracy")
for cls, acc in class_accuracies_linear.items():
    print(f"Class {cls}: {acc:.2f}%")


RBF Kernel accuracy
Class 0: 62.20%
Class 1: 66.30%
Class 2: 40.20%
Class 3: 37.10%
Class 4: 42.10%
Class 5: 43.50%
Class 6: 63.50%
Class 7: 56.90%
Class 8: 68.70%
Class 9: 60.10%
Linear Kernel accuracy
Class 0: 2.00%
Class 1: 5.80%
Class 2: 1.50%
Class 3: 4.50%
Class 4: 1.90%
Class 5: 9.70%
Class 6: 24.50%
Class 7: 26.40%
Class 8: 34.00%
Class 9: 39.60%


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing with mean and std
])

# Download and load training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# DataLoader for batching the data
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

# Define the MLP Model with One Hidden Layer
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Flatten input to 1D vector
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))  # Apply ReLU activation to hidden layer
        x = self.fc2(x)  # Output layer (no activation yet)
        return x

# Define the input dimension, hidden layer dimension, and output dimension
input_dim = 3 * 32 * 32  # CIFAR-10 images are 32x32 with 3 color channels
hidden_dim = 512  
output_dim = 10  

# Instantiate the model
model = MLP(input_dim, hidden_dim, output_dim)

# Define the Hinge Loss and Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



# Training the Model
num_epochs = 10
criterion = torch.nn.MultiMarginLoss()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}')


# Evaluate the Model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

# Initialize counters for class-wise accuracy
correct_classwise = torch.zeros(output_dim)  # To count correct predictions per class
total_classwise = torch.zeros(output_dim)    # To count total samples per class

with torch.no_grad():
    for inputs, labels in testloader:
        # Forward pass
        outputs = model(inputs)
        
        # Get predicted classes
        _, predicted = torch.max(outputs, 1)
        
        # Count correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for i in range(len(labels)):
            total_classwise[labels[i]] += 1
            correct_classwise[labels[i]] += (predicted[i] == labels[i]).item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

class_accuracies = correct_classwise / total_classwise * 100
# Print the accuracy for each class
for i in range(output_dim):
    print(f'Accuracy for class {trainset.classes[i]}: {class_accuracies[i]:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 0.4156
Epoch [2/10], Loss: 0.3264
Epoch [3/10], Loss: 0.2945
Epoch [4/10], Loss: 0.2691
Epoch [5/10], Loss: 0.2488
Epoch [6/10], Loss: 0.2325
Epoch [7/10], Loss: 0.2164
Epoch [8/10], Loss: 0.2012
Epoch [9/10], Loss: 0.1851
Epoch [10/10], Loss: 0.1757
Accuracy on the test set: 51.50%
Accuracy for class airplane: 59.50%
Accuracy for class automobile: 67.10%
Accuracy for class bird: 41.60%
Accuracy for class cat: 31.30%
Accuracy for class deer: 48.70%
Accuracy for class dog: 37.10%
Accuracy for class frog: 58.30%
Accuracy for class horse: 60.60%
Accuracy for class ship: 61.20%
Accuracy for class truck: 49.60%
